In [87]:
import numpy as np
import projectLib as lib

# set highest rating
K = 5
F = 3
eps=0.1

def softmax(x):
    # Numerically stable softmax function
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def ratingsPerMovie(training):
    movies = [x[0] for x in training]
    u_movies = np.unique(movies).tolist()
    return np.array([[i, movie, len([x for x in training if x[0] == movie])] for i, movie in enumerate(u_movies)])

def getV(ratingsForUser):
    # ratingsForUser is obtained from the ratings for user library
    # you should return a binary matrix ret of size m x K, where m is the number of movies
    #   that the user has seen. ret[i][k] = 1 if the user
    #   has rated movie ratingsForUser[i, 0] with k stars
    #   otherwise it is 0
    ret = np.zeros((len(ratingsForUser), K))
    for i in range(len(ratingsForUser)):
        ret[i, ratingsForUser[i, 1]-1] = 1.0
    return ret

def getInitialWeights(m, F, K):
    # m is the number of visible units
    # F is the number of hidden units
    # K is the highest rating (fixed to 5 here)
    return np.random.normal(0, 0.1, (m, F, K))

In [43]:
training = lib.getTrainingData()
ratingsForUser1 = lib.getRatingsForUser(1, training)

In [71]:
# ratingsPerMovie(training)
v = getV(ratingsForUser1)
# v.shape
w = getInitialWeights(v.shape[0],F,5)

In [63]:
trStats = lib.getUsefulStats(training)
W = getInitialWeights(trStats["n_movies"], F, K)

In [70]:
W[ratingsForUser1[:, 0], :, :]

array([[[-6.62179276e-02, -1.06077656e-01,  1.08694801e-01,
          1.02623931e-01,  6.76727068e-02],
        [-5.82069365e-02, -1.80134636e-01,  1.32092768e-01,
          3.38644212e-02, -9.17120948e-02],
        [ 2.79439919e-03,  1.09739335e-01,  3.87671423e-02,
         -1.75726251e-01,  1.57377998e-01]],

       [[ 2.00014631e-02, -2.63346220e-02, -1.16342054e-01,
          1.97768759e-02, -4.80679547e-02],
        [ 1.67342882e-02, -7.26718009e-02,  1.41420542e-01,
          6.32005262e-02,  7.42230677e-03],
        [ 1.74534797e-01, -1.76436747e-02,  1.21238875e-02,
         -2.25955199e-02, -1.13946065e-01]],

       [[-4.02393325e-02,  2.68471338e-01, -8.00705800e-03,
         -1.27022358e-01, -9.73222533e-03],
        [ 1.34836447e-01, -1.10560305e-02,  3.09362792e-02,
          6.44903320e-03,  6.35008250e-02],
        [-9.85657784e-04, -5.98350259e-02,  5.05700462e-02,
          5.52846046e-03, -5.09814088e-02]],

       [[ 1.99825442e-02,  1.40386722e-01,  2.22157400e-02

In [68]:
ratingsForUser1[:, 0]

array([15, 48, 19, 29, 96, 28, 23,  2])

In [177]:
def sig(x):
    ### TO IMPLEMENT ###
    # x is a real vector of size n
    # ret should be a vector of size n where ret_i = sigmoid(x_i)
    return 1/(1+np.exp(-x))

def visibleToHiddenVec(v, w):
    ### TO IMPLEMENT ###
    # v is a matrix of size m x 5. Each row is a binary vector representing a rating
    #    OR a probability distribution over the rating
    # w is a list of matrices of size m x F x 5
    # ret should be a vector of size F
    m,f,K=w.shape
    output=list()
    for i in range(f):
        summ=0
        for k in range(K):
            for j in range(m):
                summ+=v[j,k]*w[j,i,k]
        output.append(summ)
    return sig(np.array(output))
    

def hiddenToVisible(h, w):
    ### TO IMPLEMENT ###
    # h is a binary vector of size F
    # w is an array of size m x F x 5
    # ret should be a matrix of size m x 5, where m
    #   is the number of movies the user has seen.
    #   Remember that we do not reconstruct movies that the user
    #   has not rated! (where reconstructing means getting a distribution
    #   over possible ratings).
    #   We only do so when we predict the rating a user would have given to a movie.
    output=w[:,0,:]*h[0]
    m,f,k=w.shape
    for i in range(1,f):
        output+=w[:,i,:]*h[i]
    return sig(output)

def probProduct(v, p):
    # v is a matrix of size m x 5
    # p is a vector of size F, activation of the hidden units
    # returns the gradient for visible input v and hidden activations p
    ret = np.zeros((v.shape[0], p.size, v.shape[1]))
    for i in range(v.shape[0]):
        for j in range(p.size):
            for k in range(v.shape[1]):
                ret[i, j, k] = v[i, k] * p[j]
    return ret

def sample(p):
    # p is a vector of real numbers between 0 and 1
    # ret is a vector of same size as p, where ret_i = Ber(p_i)
    # In other word we sample from a Bernouilli distribution with
    # parameter p_i to obtain ret_i
    samples = np.random.random(p.size)
    return np.array(samples <= p, dtype=int)

def getPredictedDistribution(v, w, wq):
    ### TO IMPLEMENT ###
    # This function returns a distribution over the ratings for movie q, if user data is v
    # v is the dataset of the user we are predicting the movie for
    #   It is a m x 5 matrix, where m is the number of movies in the
    #   dataset of this user.
    # w is the weights array for the current user, of size m x F x 5
    # wq is the weight matrix of size F x 5 for movie q
    #   If W is the whole weights array, then wq = W[q, :, :]
    # You will need to perform the same steps done in the learning/unlearning:
    #   - Propagate the user input to the hidden units
    #   - Sample the state of the hidden units
    #   - Backpropagate these hidden states to obtain
    #       the distribution over the movie whose associated weights are wq
    # ret is a vector of size 5
    m,f,K=w.shape
    p_learn = visibleToHiddenVec(v,w)
    PG = probProduct(v, p_learn)
    hidden_activations = sample(p_learn)
#     print(hidden_activations)
    v_negative = hiddenToVisible(hidden_activations, w)
#     p_unlearn = visibleToHiddenVec(v_negative, w)
#     NG = probProduct(v_negative, p_unlearn)
# #     print(eps*(PG-NG))
# #     print (w.shape)
#     w += eps*(PG-NG)
    return v_negative

def predictRatingMax(ratingDistribution):
    ### TO IMPLEMENT ###
    # ratingDistribution is a probability distribution over possible ratings
    #   It is obtained from the getPredictedDistribution function
    # This function is one of three you are to implement
    # that returns a rating from the distribution
    # We decide here that the predicted rating will be the one with the highest probability
    result = np.where(ratingDistribution == np.amax(ratingDistribution))[0].item()
    return result+1

def predictRatingMean(ratingDistribution):
    ### TO IMPLEMENT ###
    # ratingDistribution is a probability distribution over possible ratings
    #   It is obtained from the getPredictedDistribution function
    # This function is one of three you are to implement
    # that returns a rating from the distribution
    # We decide here that the predicted rating will be the expectation over ratingDistribution
    normalized = ratingDistribution/sum(ratingDistribution)
    result = 0 
    for k in range(ratingDistribution.shape[0]):
        result += normalized[k]*(k+1)
    return result

def predictRatingExp(ratingDistribution):
    ### TO IMPLEMENT ###
    # ratingDistribution is a probability distribution over possible ratings
    #   It is obtained from the getPredictedDistribution function
    # This function is one of three you are to implement
    # that returns a rating from the distribution
    # We decide here that the predicted rating will be the expectation over
    # the softmax applied to ratingDistribution
    softmax = np.exp(ratingDistribution)/sum(np.exp(ratingDistribution))
    result = 0 
    for k in range(ratingDistribution.shape[0]):
        result += softmax[k]*(k+1)
    return result

def predictMovieForUser(q, user, W, training, predictType="exp"):
    # movie is movie idx
    # user is user ID
    # type can be "max" or "exp"
    ratingsForUser = lib.getRatingsForUser(user, training)
    v = getV(ratingsForUser)
    ratingDistribution = getPredictedDistribution(v, W[ratingsForUser[:, 0], :, :], W[q, :, :])
    if predictType == "max":
        return predictRatingMax(ratingDistribution)
    elif predictType == "mean":
        return predictRatingMean(ratingDistribution)
    else:
        return predictRatingExp(ratingDistribution)

def predict(movies, users, W, training, predictType="exp"):
    # given a list of movies and users, predict the rating for each (movie, user) pair
    # used to compute RMSE
    return [predictMovieForUser(movie, user, W, training, predictType=predictType) for (movie, user) in zip(movies, users)]

def predictForUser(user, W, training, predictType="exp"):
    ### TO IMPLEMENT
    # given a user ID, predicts all movie ratings for the user
    return None


In [179]:
q = np.random.choice(ratingsForUser1[:, 0])
index = np.where(ratingsForUser1[:, 0]==q)[0].item()
ratingdist1 = getPredictedDistribution(v, W[ratingsForUser1[:, 0], :, :], W[q, :, :])[index]
print(ratingdist1)
predictRatingExp(ratingdist1)
# Problem remains - we need the index to know which movie to take the values from in the 
# getPredictedDistribution function. Need to check with TA and clarify that part.

[0.53829413 0.55013223 0.49222976 0.46451452 0.4602866 ]


2.9516426966913345